In [5]:
import pandas as pd
import numpy as np
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import mean_absolute_error
import warnings

warnings.filterwarnings('ignore')

# %%
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')
train = train.loc[train.target != 0]

reviews_emmb = pd.read_csv('reviews_with_rubert-tiny2_name.csv').drop(columns=['name', 'pred'])
train = train.merge(reviews_emmb, on='id', how='left')
test = test.merge(reviews_emmb, on='id', how='left')

reviews_emmb = pd.read_csv('reviews_with_LaBSE-en-ru_comments.csv').drop(columns=['text'])
reviews_emmb.columns = ['labse_' + x if x != 'id' else x for x in reviews_emmb.columns]
train = train.merge(reviews_emmb, on='id', how='left')
test = test.merge(reviews_emmb, on='id', how='left')

embedding_cols = [col for col in train.columns if col.startswith('embedding_') or 'pred' in col]
for col in embedding_cols:
    train[col] = train[col].fillna(train[col].mean())
    test[col] = test[col].fillna(train[col].mean())

train['category'] = train['category'].astype('category')
test['category'] = test['category'].astype('category')

# %%
train['lat'] = train['coordinates'].apply(lambda x: eval(x)[1] if pd.notna(x) else np.nan)
train['lon'] = train['coordinates'].apply(lambda x: eval(x)[0] if pd.notna(x) else np.nan)

test['lat'] = test['coordinates'].apply(lambda x: eval(x)[1] if pd.notna(x) else np.nan)
test['lon'] = test['coordinates'].apply(lambda x: eval(x)[0] if pd.notna(x) else np.nan)

moscow_lat, moscow_lon = 55.7558, 37.6173
train['dist_from_center'] = np.sqrt((train['lat'] - moscow_lat)**2 + (train['lon'] - moscow_lon)**2)
test['dist_from_center'] = np.sqrt((test['lat'] - moscow_lat)**2 + (test['lon'] - moscow_lon)**2)

train['lat_lon_ratio'] = train['lat'] / (train['lon'] + 1e-5)
test['lat_lon_ratio'] = test['lat'] / (test['lon'] + 1e-5)

train['lat_lon_sum'] = train['lat'] + train['lon']
test['lat_lon_sum'] = test['lat'] + test['lon']

train['lat_lon_diff'] = train['lat'] - train['lon']
test['lat_lon_diff'] = test['lat'] - test['lon']

coord_features = ['lat', 'lon', 'dist_from_center', 'lat_lon_ratio', 'lat_lon_sum', 'lat_lon_diff']
for feat in coord_features:
    train[feat] = train[feat].fillna(train[feat].median())
    test[feat] = test[feat].fillna(test[feat].median())


In [6]:
exclude_cols = ['id', 'name', 'address', 'coordinates', 'target', 'text', 'name_clean', 'address_clean',
'name_language', 'address_street', 'address_building', 'address_korpus', 'address_city', 'labse_text', 'address_text']


# LightAutoML setup
task = Task('reg', loss='mae', metric='mae')

roles = {
    'target': 'target',
    'drop': [col for col in exclude_cols if col != 'target'],
}

automl = TabularAutoML(task=task, 
                       timeout=30*60, # 10 minutes
                       cpu_limit=32, # Optional: Limit CPU cores
                       reader_params={'n_jobs': 32}) # Optional: Parallelize data reading

# Train the model
oof_predictions = automl.fit_predict(train, roles=roles, verbose=2)

[18:02:26] Stdout logging level is INFO2.
[18:02:26] Task: reg

[18:02:26] Start automl preset with listed constraints:
[18:02:26] - time: 1800.00 seconds
[18:02:26] - CPU: 32 cores
[18:02:26] - memory: 16 GB

[18:02:26] Train data shape: (37167, 1373)

[18:03:05] Layer 1 train process start. Time left 1760.93 secs
[18:03:17] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[18:03:17] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[18:03:29] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[18:03:37] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[18:03:59] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[18:04:08] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[18:04:19] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -0.22831399788094303
[18:04:19] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[18:04:19] Time left 1686.72 secs

[18:05:13] Selector_LightGBM

Optimization Progress:   1%|          | 1/101 [04:19<7:12:41, 259.62s/it, best_trial=0, best_value=-0.218]

[18:12:48] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[18:12:48] The set of hyperparameters {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}
 achieve -0.2180 mae
[18:12:48] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...


[18:12:48] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM =====
[18:19:31] Time limit exceeded after calculating fold 0

[18:19:31] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = -0.21923973533393298
[18:19:31] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[18:19:32] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[18:19:32] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[18:19:46] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[18:20:00] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[18:20:14] ===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[18:20:28] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_2_CatBoost =====
[18:20:42] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = -0.2223532185881787
[18:20:42] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[18:20:42] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_

Optimization Progress:  13%|█▎        | 13/101 [03:06<21:02, 14.35s/it, best_trial=4, best_value=-0.221]

[18:23:49] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[18:23:49] The set of hyperparameters {'max_depth': 6, 'nan_mode': 'Min', 'l2_leaf_reg': 1.9826980964985924e-05, 'min_data_in_leaf': 10}
 achieve -0.2209 mae
[18:23:49] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[18:23:49] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[18:24:08] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[18:24:29] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[18:24:52] ===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[18:25:08] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[18:25:26] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = -0.22206677018571336
[18:25:26] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[18:25:26] Time left 419.72 secs

[18:25:26] Time limit exceeded in one of the tasks. AutoML will blend level 1 models.

[18:25:26] Layer 1 training completed.

[18:25:26] Blending: optimization starts with equal weights. Score = -0.2214208
[18:25:26] Blending: iteration 0: score = -0.2210053, weights = [0.05019379 0.21630998 0.46592432 0.13378595 0.13378595]
[18:25:26] Blending: no improveme

In [7]:
test_predictions = automl.predict(test)

# %%
submission = pd.DataFrame({
    'id': test['id'],
    'target': test_predictions.data[:, 0],
})

submission.to_csv('submission_lightautoml.csv', index=False)
print(f"\nSubmission file created!")

# %%
print(test_predictions)


Submission file created!
array([[3.5754457],
       [3.4182944],
       [3.6872315],
       ...,
       [3.958274 ],
       [3.4847088],
       [3.770177 ]], dtype=float32)
